<a href="https://colab.research.google.com/github/Praxis-QR/BDSN/blob/main/KK_C1_SparkSQL_SQLContext_HiveContext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://4.bp.blogspot.com/-gbL5nZDkpFQ/XScFYwoTEII/AAAAAAAAAGY/CcVb_HDLwvs2Brv5T4vSsUcz7O4r2Q79ACK4BGAYYCw/s1600/kk3-header00-beta.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

# SPARK SQL
If you wish to run HIVE natively under Hadoop please see this notebook [Hadoop and Hive](https://github.com/prithwis/KKolab/blob/main/KK_B2_Hadoop_and_Hive.ipynb)

# Install Spark

## This approach deprecated

In [ ]:
!apt-get update > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q http://apache.osuosl.org/spark/spark-2.2.2/spark-2.2.2-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
!wget -q http://apache.osuosl.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz

#
# if the current version of Spark is not used, there may be errors
# check here for current versions http://apache.osuosl.org/spark
#
#!tar xf spark-2.4.0-bin-hadoop2.7.tgz
#!tar xf spark-2.4.4-bin-hadoop2.7.tgz
#!tar xf spark-2.4.5-bin-hadoop2.7.tgz
#!tar xf spark-3.0.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
# !pip install -q findspark  -- findspark is no more required
!pip install -q pyspark
# -----------------------------------------------
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"
#import findspark
#findspark.init()
# -----------------------------------------------
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
#sc

     |████████████████████████████████| 281.3 MB 40 kB/s 
     |████████████████████████████████| 198 kB 50.3 MB/s 


## 2022 Approach

In [1]:
!pip3 -q install pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
#sc

     |████████████████████████████████| 281.4 MB 27 kB/s 
     |████████████████████████████████| 198 kB 35.0 MB/s 


# Load Data

In [2]:
# Data files downloaded from author's G-Drive
!gdown https://drive.google.com/uc?id=1JJH24ZZaiJrEKValD--UtyFcWl7UanwV  # 2% data
!gdown https://drive.google.com/uc?id=1g7mJ0v4fkERW0HWc1eq-SHs_jvQ0N2Oe  # 100% data

Downloading...
From: https://drive.google.com/uc?id=1JJH24ZZaiJrEKValD--UtyFcWl7UanwV
To: /content/eCommerce_02PC_2021.csv
100% 917k/917k [00:00<00:00, 98.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1g7mJ0v4fkERW0HWc1eq-SHs_jvQ0N2Oe
To: /content/eCommerce_Full_2021.csv
100% 45.6M/45.6M [00:00<00:00, 100MB/s] 


In [3]:
#we remove the CRLF character from the end of the row if it exists
#!sed 's/\r//' /content/eCommerce_Full_2021.csv > datafile.csv
!sed 's/\r//' /content/eCommerce_02PC_2021.csv > datafile.csv
#sed -i -e "1d" datafile.csv               # remove the first line containing headers from the file

In [4]:
data_file = 'datafile.csv'
raw_data = sc.textFile(data_file)
print ("Data Size", raw_data.count())

Data Size 10839


#Spark Data Frame

In [5]:
eCommerce_df = spark.read.csv(data_file,inferSchema=True, header=True)

In [6]:
eCommerce_df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [7]:
eCommerce_df.select('Quantity','UnitPrice','Country').groupby(eCommerce_df.Country).count().sort('count', ascending = False).show()

+--------------+-----+
|       Country|count|
+--------------+-----+
|United Kingdom| 9964|
|        France|  171|
|       Germany|  169|
|          EIRE|  157|
|       Belgium|   46|
|   Netherlands|   45|
|         Spain|   44|
|   Switzerland|   35|
|      Portugal|   26|
|        Norway|   24|
|     Australia|   18|
|       Finland|   15|
|         Italy|   14|
|       Austria|   13|
|         Japan|   12|
|   Unspecified|   11|
|        Cyprus|   10|
|        Sweden|    7|
|       Denmark|    6|
|     Hong Kong|    6|
+--------------+-----+
only showing top 20 rows



#Spark SQL
What is the difference between SQLContext and HiveContext? See [here](https://intellipaat.com/community/7599/what-is-the-difference-between-apache-spark-sqlcontext-vs-hivecontext#:~:text=HiveContext%20is%20a%20super%20set,read%20data%20from%20Hive%20tables.&text=The%20more%20basic%20SQLContext%20provides,does%20not%20depend%20on%20Hive.), or [here](https://stackoverflow.com/questions/33666545/what-is-the-difference-between-apache-spark-sqlcontext-vs-hivecontext)

## Remove comma from data of CSV file

In [8]:
#we remove the CRLF character from the end of the row if it exists
#!sed 's/\r//' /content/eCommerce_Full_2021.csv > datafile.csv
!sed 's/\r//g' /content/eCommerce_02PC_2021.csv > datafile.csv
# remove the first line containing headers from the file
!sed -i -e "1d" datafile.csv               

The data contains commas which causes havoc during the load process

In [9]:
!grep 'TROPICAL BEACH' datafile.csv

548196,82011A,"BATHROOM SCALES, TROPICAL BEACH",1,3/29/2011 15:44,3.75,16011.0,United Kingdom


In [10]:
!grep 'COFFEE,' datafile.csv

563020,21216,"SET 3 RETROSPOT TEA,COFFEE,SUGAR",4,8/11/2011 12:58,4.95,12428.0,Finland
555744,85159A,"BLACK TEA,COFFEE,SUGAR JARS",24,6/7/2011 10:44,1.95,18072.0,United Kingdom
539581,21216,"SET 3 RETROSPOT TEA,COFFEE,SUGAR",4,12/20/2010 12:51,4.95,16143.0,United Kingdom
545715,21216,"SET 3 RETROSPOT TEA,COFFEE,SUGAR",2,3/7/2011 10:13,10.79,,United Kingdom
561678,21216,"SET 3 RETROSPOT TEA,COFFEE,SUGAR",1,7/28/2011 18:33,4.95,17931.0,United Kingdom
566041,21216,"SET 3 RETROSPOT TEA,COFFEE,SUGAR",8,9/8/2011 13:48,4.95,13814.0,Germany
566073,21216,"SET 3 RETROSPOT TEA,COFFEE,SUGAR",2,9/8/2011 19:58,4.95,14562.0,United Kingdom
568712,21216,"SET 3 RETROSPOT TEA,COFFEE,SUGAR",1,9/28/2011 15:50,4.95,17841.0,United Kingdom
555327,21216,"SET 3 RETROSPOT TEA,COFFEE,SUGAR",2,6/2/2011 11:03,10.79,,United Kingdom
540093,21216,"SET 3 RETROSPOT TEA,COFFEE,SUGAR",1,1/4/2011 15:24,4.95,16725.0,United Kingdom
558538,21216,"SET 3 RETROSPOT TEA,COFFEE,SUGAR",2,6/30/2011 12:12,4.95,15005.0,United Kingdom


In [11]:
!head datafile.csv

C544414,22960,JAM MAKING SET WITH JARS,-2,2/18/2011 14:54,3.75,13408.0,United Kingdom
555276,48111,DOORMAT 3 SMILEY CATS,1,6/1/2011 17:28,15.79,,United Kingdom
575656,22952,60 CAKE CASES VINTAGE CHRISTMAS,48,11/10/2011 14:29,0.55,13319.0,United Kingdom
571636,20674,GREEN POLKADOT BOWL,16,10/18/2011 11:41,1.25,13509.0,United Kingdom
576657,22556,PLASTERS IN TIN CIRCUS PARADE ,12,11/16/2011 11:03,1.65,12720.0,Germany
569823,23298,SPOTTY BUNTING,1,10/6/2011 12:15,4.95,16895.0,United Kingdom
570185,21090,WET/MOULDY,-192,10/7/2011 14:56,0.0,,United Kingdom
574943,17091J,VANILLA INCENSE IN TIN,36,11/8/2011 7:52,0.38,13026.0,United Kingdom
561902,22909,SET OF 20 VINTAGE CHRISTMAS NAPKINS,96,7/31/2011 15:50,0.85,17404.0,Sweden
550499,22138,BAKING SET 9 PIECE RETROSPOT ,1,4/18/2011 15:16,10.79,,United Kingdom


commas getting removed here

In [13]:
import pandas as pd

In [14]:
#p_df = pd.read_csv('datafile.csv',header=None)   # throws errors because of non utf-encoding 
                                                  # https://stackoverflow.com/questions/18171739/unicodedecodeerror-when-reading-csv-file-in-pandas-with-python
p_df = pd.read_csv('datafile.csv',encoding = "ISO-8859-1",header=None)
#p_df = pd.read_csv('datafile.csv',encoding = "latin",header=None)
#p_df = pd.read_csv('datafile.csv',encoding = "latin")
p_df2 = p_df.replace(',', ' ', regex=True)

In [15]:
p_df2.shape

(10838, 8)

In [13]:
#Desc2 = p_df2['Description'].str.encode('ascii','ignore').str.decode('ascii')

In [14]:
#p_df2['Description2'] = Desc2

In [16]:
#p_df2.head(3)

In [17]:
#p_df2.drop("Description", axis=1, inplace=True)

In [18]:
#p_df2.head(3)

In [19]:
p_df2.to_csv('datafile2.csv',index=False,sep=',',header=False)

In [20]:
!head datafile2.csv

C544414,22960,JAM MAKING SET WITH JARS,-2,2/18/2011 14:54,3.75,13408.0,United Kingdom
555276,48111,DOORMAT 3 SMILEY CATS,1,6/1/2011 17:28,15.79,,United Kingdom
575656,22952,60 CAKE CASES VINTAGE CHRISTMAS,48,11/10/2011 14:29,0.55,13319.0,United Kingdom
571636,20674,GREEN POLKADOT BOWL,16,10/18/2011 11:41,1.25,13509.0,United Kingdom
576657,22556,PLASTERS IN TIN CIRCUS PARADE ,12,11/16/2011 11:03,1.65,12720.0,Germany
569823,23298,SPOTTY BUNTING,1,10/6/2011 12:15,4.95,16895.0,United Kingdom
570185,21090,WET/MOULDY,-192,10/7/2011 14:56,0.0,,United Kingdom
574943,17091J,VANILLA INCENSE IN TIN,36,11/8/2011 7:52,0.38,13026.0,United Kingdom
561902,22909,SET OF 20 VINTAGE CHRISTMAS NAPKINS,96,7/31/2011 15:50,0.85,17404.0,Sweden
550499,22138,BAKING SET 9 PIECE RETROSPOT ,1,4/18/2011 15:16,10.79,,United Kingdom


In [21]:
!grep 'TROPICAL BEACH' datafile2.csv

548196,82011A,BATHROOM SCALES  TROPICAL BEACH,1,3/29/2011 15:44,3.75,16011.0,United Kingdom


In [22]:
!grep 'COFFEE' datafile2.csv

539089,22303,COFFEE MUG APPLES DESIGN,6,12/16/2010 9:46,2.55,15246.0,United Kingdom
539744,22300,COFFEE MUG DOG + BALL DESIGN,6,12/21/2010 15:21,5.06,,United Kingdom
538349,22301,COFFEE MUG CAT + BIRD DESIGN,2,12/10/2010 14:59,5.06,,United Kingdom
557570,22301,COFFEE MUG CAT + BIRD DESIGN,2,6/21/2011 10:48,2.55,15998.0,United Kingdom
571965,22971,QUEENS GUARD COFFEE MUG,2,10/20/2011 10:41,2.55,17238.0,United Kingdom
562772,37370,RETRO COFFEE MUGS ASSORTED,6,8/9/2011 12:07,1.25,13097.0,United Kingdom
548014,35810B,ENAMEL BLUE RIM COFFEE CONTAINER,12,3/29/2011 11:29,0.83,17719.0,United Kingdom
574038,23243,SET OF TEA COFFEE SUGAR TINS PANTRY,4,11/2/2011 12:56,4.95,18164.0,United Kingdom
563020,21216,SET 3 RETROSPOT TEA COFFEE SUGAR,4,8/11/2011 12:58,4.95,12428.0,Finland
561926,23243,SET OF TEA COFFEE SUGAR TINS PANTRY,4,8/1/2011 12:48,4.95,12749.0,United Kingdom
555744,85159A,BLACK TEA COFFEE SUGAR JARS,24,6/7/2011 10:44,1.95,18072.0,United Kingdom
561515,22970,LONDON BUS COFFEE MUG,2,7/

##SQL Context

In [23]:
#from pyspark.sql import SQLContext
from pyspark.sql import Row

#sqlContext = SQLContext(sc)

In [24]:
data_file = "datafile2.csv"
raw_data = sc.textFile(data_file)
csv_data = raw_data.map(lambda l: l.split(","))

In [25]:
csv_data.take(5)

[['C544414',
  '22960',
  'JAM MAKING SET WITH JARS',
  '-2',
  '2/18/2011 14:54',
  '3.75',
  '13408.0',
  'United Kingdom'],
 ['555276',
  '48111',
  'DOORMAT 3 SMILEY CATS',
  '1',
  '6/1/2011 17:28',
  '15.79',
  '',
  'United Kingdom'],
 ['575656',
  '22952',
  '60 CAKE CASES VINTAGE CHRISTMAS',
  '48',
  '11/10/2011 14:29',
  '0.55',
  '13319.0',
  'United Kingdom'],
 ['571636',
  '20674',
  'GREEN POLKADOT BOWL',
  '16',
  '10/18/2011 11:41',
  '1.25',
  '13509.0',
  'United Kingdom'],
 ['576657',
  '22556',
  'PLASTERS IN TIN CIRCUS PARADE ',
  '12',
  '11/16/2011 11:03',
  '1.65',
  '12720.0',
  'Germany']]

In [26]:
row_data = csv_data.map(lambda p: Row(
    InvoiceNo=p[0], 
    StockCode=p[1],
    Description=p[2],
    Quantity=int(p[3]),
    InvoiceDate=p[4],
    UnitPrice=float(p[5]),
    CustomerID=p[6],
    Country=p[7]
    )
)

In [27]:
row_data.take(2)

[Row(InvoiceNo='C544414', StockCode='22960', Description='JAM MAKING SET WITH JARS', Quantity=-2, InvoiceDate='2/18/2011 14:54', UnitPrice=3.75, CustomerID='13408.0', Country='United Kingdom'),
 Row(InvoiceNo='555276', StockCode='48111', Description='DOORMAT 3 SMILEY CATS', Quantity=1, InvoiceDate='6/1/2011 17:28', UnitPrice=15.79, CustomerID='', Country='United Kingdom')]

In [28]:
#eCommerce2_df = sqlContext.createDataFrame(row_data)
#eCommerce2_df.registerTempTable("eCommerce")
# spark 2 onwards ...
eCommerce2_df = spark.createDataFrame(row_data)
eCommerce2_df.createOrReplaceTempView("eCommerce")

In [29]:
#Countries = sqlContext.sql("SELECT * FROM eCommerce limit 20")
Countries = spark.sql("SELECT * FROM eCommerce limit 20")
Countries.show()

+---------+---------+--------------------+--------+----------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|     InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+----------------+---------+----------+--------------+
|  C544414|    22960|JAM MAKING SET WI...|      -2| 2/18/2011 14:54|     3.75|   13408.0|United Kingdom|
|   555276|    48111|DOORMAT 3 SMILEY ...|       1|  6/1/2011 17:28|    15.79|          |United Kingdom|
|   575656|    22952|60 CAKE CASES VIN...|      48|11/10/2011 14:29|     0.55|   13319.0|United Kingdom|
|   571636|    20674| GREEN POLKADOT BOWL|      16|10/18/2011 11:41|     1.25|   13509.0|United Kingdom|
|   576657|    22556|PLASTERS IN TIN C...|      12|11/16/2011 11:03|     1.65|   12720.0|       Germany|
|   569823|    23298|      SPOTTY BUNTING|       1| 10/6/2011 12:15|     4.95|   16895.0|United Kingdom|
|   570185|    21090|          WET/MOULDY|    -192| 10/

In [30]:
#Countries = sqlContext.sql("SELECT sum(Quantity), sum(UnitPrice), Country from eCommerce group by Country order by sum(Quantity) desc")
Countries = spark.sql("SELECT sum(Quantity), sum(UnitPrice), Country from eCommerce group by Country order by sum(Quantity) desc")
Countries.show()

+-------------+------------------+---------------+
|sum(Quantity)|    sum(UnitPrice)|        Country|
+-------------+------------------+---------------+
|        88067| 44191.70000000067| United Kingdom|
|         3762|             86.12|    Netherlands|
|         2725| 525.6800000000001|         France|
|         2266| 680.6699999999995|           EIRE|
|         1758| 567.3799999999997|        Germany|
|         1567| 54.92999999999999|      Australia|
|          735|28.759999999999998|          Japan|
|          520|179.85000000000005|        Belgium|
|          405|143.37000000000003|    Switzerland|
|          385|             49.47|         Sweden|
|          362|            135.62|          Spain|
|          319|            106.17|         Norway|
|          181|            471.33|       Portugal|
|          179|             75.02|        Finland|
|          144|              40.9|      Hong Kong|
|          127|           2065.98|      Singapore|
|          118|             12.

## HIVE context
this part adapted from [this notebook](https://colab.research.google.com/github/jmbanda/BigDataProgramming_2019/blob/master/Chapter_5_Loading_and_Saving_Data_in_Spark.ipynb#scrollTo=lOcCViaN4cwC)

In [31]:
from pyspark.sql import Row
from pyspark.sql import HiveContext
sqlContext = HiveContext(sc)


/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:604: FutureWarning: HiveContext is deprecated in Spark 2.0.0. Please use SparkSession.builder.enableHiveSupport().getOrCreate() instead.
  FutureWarning


In [32]:
data_file = "datafile2.csv"
raw_data = sc.textFile(data_file)
csv_data = raw_data.map(lambda l: l.split(","))

In [33]:
row_data = csv_data.map(lambda p: Row(
    InvoiceNo=p[0], 
    StockCode=p[1],
    Description=p[2],
    Quantity=int(p[3]),
    InvoiceDate=p[4],
    UnitPrice=float(p[5]),
    CustomerID=p[6],
    Country=p[7]
    )
)

In [34]:
ec_schema = spark.createDataFrame(row_data)
# Register it as a temp table
sqlContext.registerDataFrameAsTable(ec_schema, "ec_HiveTable")
sqlContext.sql("show tables").show()

+---------+------------+-----------+
|namespace|   tableName|isTemporary|
+---------+------------+-----------+
|         |ec_hivetable|       true|
|         |   ecommerce|       true|
+---------+------------+-----------+



In [35]:
Countries = sqlContext.sql("SELECT * FROM ec_HiveTable limit 20")
Countries.show()

+---------+---------+--------------------+--------+----------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|     InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+----------------+---------+----------+--------------+
|  C544414|    22960|JAM MAKING SET WI...|      -2| 2/18/2011 14:54|     3.75|   13408.0|United Kingdom|
|   555276|    48111|DOORMAT 3 SMILEY ...|       1|  6/1/2011 17:28|    15.79|          |United Kingdom|
|   575656|    22952|60 CAKE CASES VIN...|      48|11/10/2011 14:29|     0.55|   13319.0|United Kingdom|
|   571636|    20674| GREEN POLKADOT BOWL|      16|10/18/2011 11:41|     1.25|   13509.0|United Kingdom|
|   576657|    22556|PLASTERS IN TIN C...|      12|11/16/2011 11:03|     1.65|   12720.0|       Germany|
|   569823|    23298|      SPOTTY BUNTING|       1| 10/6/2011 12:15|     4.95|   16895.0|United Kingdom|
|   570185|    21090|          WET/MOULDY|    -192| 10/

In [36]:
Countries = sqlContext.sql("SELECT sum(Quantity), sum(UnitPrice), Country from eCommerce group by Country order by sum(Quantity) desc")
Countries.show()

+-------------+------------------+---------------+
|sum(Quantity)|    sum(UnitPrice)|        Country|
+-------------+------------------+---------------+
|        88067| 44191.70000000067| United Kingdom|
|         3762|             86.12|    Netherlands|
|         2725| 525.6800000000001|         France|
|         2266| 680.6699999999995|           EIRE|
|         1758| 567.3799999999997|        Germany|
|         1567| 54.92999999999999|      Australia|
|          735|28.759999999999998|          Japan|
|          520|179.85000000000005|        Belgium|
|          405|143.37000000000003|    Switzerland|
|          385|             49.47|         Sweden|
|          362|            135.62|          Spain|
|          319|            106.17|         Norway|
|          181|            471.33|       Portugal|
|          179|             75.02|        Finland|
|          144|              40.9|      Hong Kong|
|          127|           2065.98|      Singapore|
|          118|             12.

#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself. 
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)